In [1]:
!pip install transformers==4.15.0
!pip install tensorboardX

     |████████████████████████████████| 3.4 MB 4.6 MB/s 
     |████████████████████████████████| 895 kB 22.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 61 kB 406 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 124 kB 5.1 MB/s 


In [2]:
%cd drive/My\ Drive/Indenpedent\ Study

/content/drive/My Drive/Indenpedent Study


In [ ]:
!python prepare_data.py --data_dir data/sis/ --out_dir data/sis/ --task_name sind

100 990
200 1990
300 2990
400 3990
500 4990
600 5990
700 6990
800 7990
900 8990
1000 9990
1100 10990
1200 11990
1300 12990
1400 13983
1500 14977
1600 15977
1700 16977
1800 17977
1900 18977
2000 19977
2100 20977
2200 21977
2300 22977
2400 23977
2500 24975
2600 25975
2700 26975
2800 27974
2900 28974
3000 29974
3100 30974
3200 31974
3300 32974
3400 33974
3500 34974
3600 35974
3700 36974
3800 37966
3900 38966
4000 39966
4100 40966
4200 41966
4300 42966
4400 43966
4500 44966
4600 45966
4700 46966
4800 47966
4900 48965
5000 49965
5100 50965
5200 51964
5300 52964
5400 53964
5500 54964
5600 55964
5700 56964
5800 57964
5900 58964
6000 59964
6100 60964
6200 61964
6300 62964
6400 63964
6500 64964
6600 65964
6700 66964
6800 67964
6900 68964
7000 69964
7100 70964
7200 71964
7300 72964
7400 73964
7500 74964
7600 75964
7700 76964
7800 77964
7900 78964
8000 79964
8100 80964
8200 81964
8300 82964
8400 83964
8500 84964
8600 85964
8700 86964
8800 87964
8900 88964
9000 89964
9100 90964
9200 91964
9300 929

In [31]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset, Dataset)
from tensorboardX import SummaryWriter
from tqdm import tqdm, trange
from transformers import BertTokenizer, BertPreTrainedModel, BertModel
from transformers.modeling_outputs import  NextSentencePredictorOutput
from transformers import AdamW, get_linear_schedule_with_warmup

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers import DataProcessor, InputExample, InputFeatures

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    

In [4]:
class BertOnlyNSPHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        return seq_relationship_score
        
class BertForSentenceOrder(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.cls = BertOnlyNSPHead(config)

        # Initialize weights and apply final processing
        #self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        **kwargs,
    ):
        if "next_sentence_label" in kwargs:
            warnings.warn(
                "The `next_sentence_label` argument is deprecated and will be removed in a future version, use `labels` instead.",
                FutureWarning,
            )
            labels = kwargs.pop("next_sentence_label")

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        seq_relationship_scores = self.cls(pooled_output)

        next_sentence_loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            next_sentence_loss = loss_fct(seq_relationship_scores.view(-1, 2), labels.view(-1))

        if not return_dict:
            output = (seq_relationship_scores,) + outputs[2:]
            return ((next_sentence_loss,) + output) if next_sentence_loss is not None else output

        return NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [5]:
model = BertForSentenceOrder.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/Indenpedent Study/output/model.pt'))
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSentenceOrder: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSentenceOrder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSentenceOrder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [10]:
class PairProcessor(DataProcessor):
    """Pair Processor for the pair ordering task."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")
    
    def get_test_examples(self, data_dir):
        return self._create_test_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")
    
    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (_, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, line[0])
            try:
                text_a = line[1].lower()
                text_b = line[2].lower()
                label = line[3]
            except IndexError:
                print('cannot read the line: ' + line)
                continue
            examples.append(InputExample(
                                        guid=guid, 
                                        text_a=text_a, 
                                        text_b=text_b, 
                                        label=label
                                    ))
        return examples
    
    def _create_test_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples, rows = [], []
        for (_, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, line[0])
            try:
                text_a = line[1].lower()
                text_b = line[2].lower()
                label = line[3]
            except IndexError:
                print('cannot read the line: ' + line)
                continue
            examples.append(InputExample(
                                    guid=guid, 
                                    text_a=text_a, 
                                    text_b=text_b, 
                                    label=label
                            ))
            rows.append(line)
        return examples, rows

In [16]:
processor = PairProcessor()
label_list = processor.get_labels()
examples = processor.get_train_examples('data/sis/')
features = convert_examples_to_features(examples,
                            tokenizer,
                            label_list=label_list,
                            max_length=105,
                            output_mode='classification',
                            )
torch.save(features, 'data/sis/cached_train_bert_pair_order')

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [17]:
all_input_ids = torch.tensor(
    [f.input_ids for f in features], dtype=torch.long)
all_attention_mask = torch.tensor(
    [f.attention_mask for f in features], dtype=torch.long)
all_token_type_ids = torch.tensor(
    [f.token_type_ids for f in features], dtype=torch.long)

all_labels = torch.tensor(
    [f.label for f in features], dtype=torch.long)

train_dataset = TensorDataset(
    all_input_ids, all_attention_mask, all_token_type_ids, all_labels)


In [28]:
import logging
import random
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
train_batch_size = 32
train_sampler = RandomSampler(train_dataset) 
train_dataloader = DataLoader(    
    train_dataset, 
    sampler=train_sampler, 
    batch_size=train_batch_size
    )

t_total = len(train_dataloader)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 
    'weight_decay': 0.0
    },
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
    'weight_decay': 0.0
    }
    ]
optimizer = AdamW(
    optimizer_grouped_parameters, 
    lr=5e-5, 
    eps=1e-8,
    )
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=t_total
    )

# Train!
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_dataset))
logger.info("  Num Epochs = %d", 3)
logger.info("  Instantaneous batch size per GPU = %d", 1)
logger.info("  Total optimization steps = %d", t_total)

global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
train_iterator = trange(1, desc="Epoch")
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
for _ in train_iterator:
  epoch_iterator = tqdm(train_dataloader, desc="Iteration")
  for step, batch in enumerate(epoch_iterator):
      model.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]
              }

      outputs = model(**inputs)
      loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      tr_loss += loss.item()
      optimizer.step()
      scheduler.step()  # Update learning rate schedule
      model.zero_grad()
      global_step += 1


      if global_step % 2000 == 0:
        # Save model checkpoint
        output_dir = os.path.join('sind_bert/checkpoint-{}'.format(global_step))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)          
        model.save_pretrained(output_dir)
        logger.info("Saving model checkpoint to %s", output_dir)


logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


Streaming output truncated to the last 5000 lines.
Iteration:  32%|███▏      | 7999/25092 [2:23:29<5:05:50,  1.07s/it]01/11/2022 04:07:09 - INFO - __main__ -   Saving model checkpoint to sind_bert/checkpoint-8000

Iteration:  40%|███▉      | 9999/25092 [2:59:16<4:29:35,  1.07s/it]01/11/2022 04:42:56 - INFO - __main__ -   Saving model checkpoint to sind_bert/checkpoint-10000

Iteration:  44%|████▍     | 11160/25092 [3:20:03<4:09:58,  1.08s/it]

In [36]:
!python model.py --data_dir data/sis/ --output_dir sind_bert/checkpoint-2000/ --do_test --per_gpu_eval_batch_size 16

01/11/2022 01:43:01 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

In [12]:
!python topological_sort.py --file_path data/sis/test_results.tsv


Perfect Match: 20.59347181008902
Sentence Accuracy: 52.2809099901088
Rouge-S: 78.37586547972305
LCS: 77.19090009891197
Kendall Tau Ratio: 0.6007121661721088
Min Dist Metric for window 1: 82.64688427299703
Min Dist Metric for window 2: 94.9515331355094
Min Dist Metric for window 3: 99.07814045499505
